In [37]:
import pandas as pd
from glob import glob
from experiments_to_run import *

from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    mean_absolute_percentage_error
)

def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)


def normalized_root_mean_squared_error(y_true, y_pred, norm_factor=None):
    if norm_factor is None:
        assert False, "Set norm_factor (for example the average target value for the training set)"
    rmse = root_mean_squared_error(y_true, y_pred)
    return (rmse / norm_factor)*100

In [38]:
paths = glob("../results/metrics*")

df = []
for path in paths:
    tmp_df = pd.read_pickle(path)
    df.append(tmp_df)
df = pd.concat(df)
df

,target,config,model_name,model,hyperparams,fold,model_obj,MSE,R2,MAPE,RMSE,NRMSE
0,CSE,Conf3,SVR,SVR,"{'gamma': 1e-05, 'C': 10}",0,"SVR(C=10, gamma=1e-05)",1.018906,0.264363,3.103306,1.009409,59.885845
1,CSE,Conf3,SVR,SVR,"{'gamma': 1e-05, 'C': 10}",1,"SVR(C=10, gamma=1e-05)",1.131571,0.159466,2.034741,1.063753,63.559086
2,CSE,Conf3,SVR,SVR,"{'gamma': 1e-05, 'C': 10}",2,"SVR(C=10, gamma=1e-05)",1.653105,0.117658,1.132019,1.285731,78.181357
3,CSE,Conf3,SVR,SVR,"{'gamma': 1e-05, 'C': 10}",3,"SVR(C=10, gamma=1e-05)",1.305581,-0.072930,1.036159,1.142620,65.509885
4,CSE,Conf3,SVR,SVR,"{'gamma': 1e-05, 'C': 10}",4,"SVR(C=10, gamma=1e-05)",1.060417,0.199509,0.635240,1.029765,61.945696
...,...,...,...,...,...,...,...,...,...,...,...,...
0,CS,Conf1,MLPRegressor,MLP,"{'max_iter': 20, 'learning_rate_init': 0.01, '...",0,"MLPRegressor(batch_size=10, hidden_layer_sizes...",4579.756495,-0.006206,7.173968,67.673898,90.301721
1,CS,Conf1,MLPRegressor,MLP,"{'max_iter': 20, 'learning_rate_init': 0.01, '...",1,"MLPRegressor(batch_size=10, hidden_layer_sizes...",2385.390950,0.187266,1.720632,48.840464,62.557296
2,CS,Conf1,MLPRegressor,MLP,"{'max_iter': 20, 'learning_rate_init': 0.01, '...",2,"MLPRegressor(batch_size=10, hidden_layer_sizes...",5440.997113,-0.270485,6.710148,73.763115,95.923982
3,CS,Conf1,MLPRegressor,MLP,"{'max_iter': 20, 'learning_rate_init': 0.01, '...",3,"MLPRegressor(batch_size=10, hidden_layer_sizes...",2821.396669,0.129511,1.523890,53.116821,68.889693


In [41]:
ENSEMBLE_MODELS = [
    "RF",
    "CatBoost",
    "LightGBM",
    "AdaBoost",
]

ensemble_res = []

for TARGET in df.target.unique():
    for CONFIG in df.config.unique():
        
        root = "../results"
        res = None
        
        print("="*20)
        print(TARGET, "|", CONFIG)
        
        for MODEL in ENSEMBLE_MODELS:

            data = pd.read_pickle(f"{root}/predictions--{TARGET}--{CONFIG}--{MODEL}.pickle")
            if res is None:
                res = data[["y_train", "y_test", "y_pred"]].copy()
            else:
                res += data[["y_train", "y_test", "y_pred"]]
            data["R2"] = data.apply(lambda row: r2_score(row.y_test, row.y_pred), axis=1)
            data["RMSE"] = data.apply(lambda row: root_mean_squared_error(row.y_test, row.y_pred), axis=1)


            data["MSE"] = data.apply(lambda row: mean_squared_error(row.y_test, row.y_pred), axis=1)
            data["R2"] = data.apply(lambda row: r2_score(row.y_test, row.y_pred), axis=1)
            data["MAPE"] = data.apply(lambda row: mean_absolute_percentage_error(row.y_test, row.y_pred), axis=1)
            data["RMSE"] = data.apply(lambda row: root_mean_squared_error(row.y_test, row.y_pred), axis=1)
            data["NRMSE"] = data.apply(lambda row: normalized_root_mean_squared_error(row.y_test, row.y_pred, norm_factor=row.y_train.mean()), axis=1)


            # print(MODEL)
            # print(data.R2.mean().round(2), "+/-", data.R2.std().round(2))
            # print(data.RMSE.mean().round(2), "+/-", data.RMSE.std().round(2))
            # print()

        res = res/len(ENSEMBLE_MODELS)

        res["MSE"] = res.apply(lambda row: mean_squared_error(row.y_test, row.y_pred), axis=1)
        res["R2"] = res.apply(lambda row: r2_score(row.y_test, row.y_pred), axis=1)
        res["MAPE"] = res.apply(lambda row: mean_absolute_percentage_error(row.y_test, row.y_pred), axis=1)
        res["RMSE"] = res.apply(lambda row: root_mean_squared_error(row.y_test, row.y_pred), axis=1)
        res["NRMSE"] = res.apply(lambda row: normalized_root_mean_squared_error(row.y_test, row.y_pred, norm_factor=row.y_train.mean()), axis=1)
        res["target"] = TARGET
        res["config"] = CONFIG
        res["model"] = "▸ Ensemble"
        res["fold"] = data["fold"]
        
        ensemble_res.append(res)

        # print("Ensemble")
        # print(res.R2.mean().round(2), "+/-", res.R2.std().round(2))
        # print(res.RMSE.mean().round(2), "+/-", res.RMSE.std().round(2))
        # print()

CSE | Conf3
CSE | Conf2
CSE | Conf1
CSE | Conf4
CS | Conf3
CS | Conf2
CS | Conf1
CS | Conf4


In [42]:
df_ens = pd.concat(ensemble_res)
df_ens

,y_train,y_test,y_pred,MSE,R2,MAPE,RMSE,NRMSE,target,config,model,fold
0,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[2.308917969, 2.761816837, 2.065990209, 4.8443...","[2.8223076037090817, 1.0132061081274157, 1.079...",0.676113,0.511855,1.297171,0.822261,48.782822,CSE,Conf3,▸ Ensemble,0
1,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[0.211443918, 1.135178684, 2.124200655, 1.3913...","[1.6073268137188899, 1.7700441840760193, 2.142...",0.612880,0.544751,1.735181,0.782867,46.776152,CSE,Conf3,▸ Ensemble,1
2,"[0.641463102, 1.76086007, 0.211443918, 2.30891...","[0.068318156, 2.333114009, 1.121438265, 0.7822...","[0.7648832482216044, 2.240633225383995, 1.2687...",0.972125,0.481130,1.415887,0.985964,59.953431,CSE,Conf3,▸ Ensemble,2
3,"[0.068318156, 2.333114009, 0.211443918, 2.3089...","[0.641463102, 1.76086007, 2.918196043, 0.70906...","[0.8920574793392542, 2.5572446742548545, 2.369...",1.007585,0.171964,0.830502,1.003785,57.550055,CSE,Conf3,▸ Ensemble,3
4,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[2.695384447, 0.980406335, 5.341723493, 1.4675...","[3.1801751759149917, 2.1476138449600217, 3.532...",0.736684,0.443890,0.690826,0.858303,51.631333,CSE,Conf3,▸ Ensemble,4
0,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[2.308917969, 2.761816837, 2.065990209, 4.8443...","[2.6226345235504906, 1.5765935796709043, 1.400...",0.921546,0.334655,2.304135,0.959972,56.952882,CSE,Conf2,▸ Ensemble,0
1,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[0.211443918, 1.135178684, 2.124200655, 1.3913...","[2.883777429993599, 1.7186414649522586, 1.4311...",0.937636,0.303521,2.350511,0.968316,57.856746,CSE,Conf2,▸ Ensemble,1
2,"[0.641463102, 1.76086007, 0.211443918, 2.30891...","[0.068318156, 2.333114009, 1.121438265, 0.7822...","[1.1025363238572274, 2.2265139686869757, 1.197...",1.242088,0.337037,1.856233,1.114490,67.768721,CSE,Conf2,▸ Ensemble,2
3,"[0.068318156, 2.333114009, 0.211443918, 2.3089...","[0.641463102, 1.76086007, 2.918196043, 0.70906...","[1.1467667960836747, 2.531071366337714, 1.8350...",0.893576,0.265657,0.900524,0.945292,54.196429,CSE,Conf2,▸ Ensemble,3
4,"[0.641463102, 1.76086007, 0.068318156, 2.33311...","[2.695384447, 0.980406335, 5.341723493, 1.4675...","[2.7280892061249107, 2.517232010119568, 2.4364...",1.035344,0.218436,0.857215,1.017519,61.208980,CSE,Conf2,▸ Ensemble,4


In [47]:
metrics = ["R2", "MAPE", "RMSE", "NRMSE"]

mean_df = df.groupby(["target", "config", "model"]).agg("mean")[metrics]
std_df = df.groupby(["target", "config", "model"]).agg("std")[metrics]

mean_df_ens = df_ens.groupby(["target", "config", "model"]).agg("mean")[metrics]
std_df_ens = df_ens.groupby(["target", "config", "model"]).agg("std")[metrics]

for target in mean_df.index.levels[0]:
    for config in mean_df.index.levels[1]:
        
        # add Average model performance
        item = pd.DataFrame(
            mean_df.loc[target, config][metrics].mean().to_dict(),
            index=[(target, config, "▸ Average")]
        )
        mean_df = pd.concat((mean_df, item))

for target in std_df.index.levels[0]:
    for config in std_df.index.levels[1]:
        
        # add Average model performance
        item = pd.DataFrame(
            std_df.loc[target, config][metrics].mean().to_dict(),
            index=[(target, config, "▸ Average")]
        )
        std_df = pd.concat((std_df, item))

mean_df = pd.concat([mean_df, mean_df_ens])
std_df = pd.concat([std_df, std_df_ens])
        
display(mean_df)
display(std_df)

/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_dict(),
/tmp/ipykernel_17101/4145601540.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  mean_df.loc[target, config][metrics].mean().to_

R2      MAPE       RMSE      NRMSE
target config model                                               
CS     Conf1  AdaBoost    0.255496  2.184462  52.610980  68.581408
              BaggedDT    0.299655  2.479573  51.014196  66.491616
              BayesianNN  0.126449  3.955176  57.218001  74.596919
              CatBoost    0.323252  2.513251  50.166156  65.397953
              GBDT        0.307899  2.424466  50.810697  66.233956
...                            ...       ...        ...        ...
       Conf4  ▸ Ensemble  0.718164  0.906665  32.160359  41.935186
CSE    Conf1  ▸ Ensemble  0.285022  1.614130   1.004661  59.796315
       Conf2  ▸ Ensemble  0.291861  1.653724   1.001118  59.596752
       Conf3  ▸ Ensemble  0.430718  1.193913   0.890636  52.938759
       Conf4  ▸ Ensemble  0.436906  1.204378   0.886102  52.669168

[112 rows x 4 columns]

R2      MAPE      RMSE     NRMSE
target config model                                             
CS     Conf1  AdaBoost    0.089411  1.513082  4.594950  6.956043
              BaggedDT    0.079249  1.741851  4.040796  6.182929
              BayesianNN  0.033326  3.260382  5.968229  8.763668
              CatBoost    0.073120  1.878766  4.105460  6.388449
              GBDT        0.048208  1.761562  4.217769  6.457731
...                            ...       ...       ...       ...
       Conf4  ▸ Ensemble  0.068467  0.511238  4.168277  6.014804
CSE    Conf1  ▸ Ensemble  0.070329  0.692335  0.062331  4.739102
       Conf2  ▸ Ensemble  0.050215  0.733364  0.068934  5.209643
       Conf3  ▸ Ensemble  0.149378  0.429555  0.099027  5.644213
       Conf4  ▸ Ensemble  0.139826  0.446077  0.088783  5.004521

[112 rows x 4 columns]

In [48]:
for target_var in TARGETS:
    
    tmp = df[(df.target == target_var)]\
      [["target", "config", "model", "hyperparams"]]\
      .groupby(["config", "model"]).agg({
        "hyperparams":"first",
      }).reset_index()

    tmp.hyperparams = tmp.hyperparams.apply(
        lambda x: "\n".join([f"{k}: {v}" for k,v in x.items()]) if x is not None else ""
    )
    
    gb = tmp.groupby("model")
    new_tmp = []
    for m,g in gb:
        new_tmp.append(g.drop(columns=["model"]).rename(columns={"hyperparams":m}).set_index("config").T)
    
    tmp = pd.concat(new_tmp).reset_index().rename(columns={"index":"Model"})
    
    print("     "+target_var)
    print("results exported to", f"../results/hyperparams_{target_var}.[txt/xlsx]")
    tmp.to_markdown(f"../results/hyperparams_{target_var}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../results/hyperparams_{target_var}.xlsx", index=False)
    
    print("results exported to", f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.[txt/xlsx/csv]")
    tmp.to_markdown(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.xlsx", index=False)
    tmp.to_csv(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.csv", index=False)


     CS
results exported to ../results/hyperparams_CS.[txt/xlsx]
results exported to ../figures_and_tables/table_appendix_hyperparameters_CS.[txt/xlsx/csv]
     CSE
results exported to ../results/hyperparams_CSE.[txt/xlsx]
results exported to ../figures_and_tables/table_appendix_hyperparameters_CSE.[txt/xlsx/csv]


In [49]:
tmp

config,Model,Conf1,Conf2,Conf3,Conf4
0,AdaBoost,n_estimators: 50\nlearning_rate: 1.0,n_estimators: 100\nlearning_rate: 1.0,n_estimators: 100\nlearning_rate: 1.0,n_estimators: 200\nlearning_rate: 1.0
1,BaggedDT,n_estimators: 200\nbase_estimator__max_depth: 5,n_estimators: 200\nbase_estimator__max_depth: 4,n_estimators: 200\nbase_estimator__max_depth: 10,n_estimators: 200\nbase_estimator__max_depth: 4
2,BayesianNN,n_iter: 100\nlambda_2: 0.0001\nlambda_1: 1e-06...,n_iter: 300\nlambda_2: 0.0001\nlambda_1: 0.000...,n_iter: 100\nlambda_2: 0.0001\nlambda_1: 1e-06...,n_iter: 300\nlambda_2: 0.0001\nlambda_1: 0.000...
3,CatBoost,learning_rate: 0.03\nl2_leaf_reg: 1\niteration...,learning_rate: 0.03\nl2_leaf_reg: 1\niteration...,learning_rate: 0.1\nl2_leaf_reg: 1\niterations...,learning_rate: 0.03\nl2_leaf_reg: 1\niteration...
4,GBDT,subsample: 1\nn_estimators: 200\nmax_leaf_node...,subsample: 1\nn_estimators: 1000\nmax_leaf_nod...,subsample: 1\nn_estimators: 200\nmax_leaf_node...,subsample: 1\nn_estimators: 200\nmax_leaf_node...
5,KNN,n_neighbors: 8,n_neighbors: 10,n_neighbors: 10,n_neighbors: 7
6,LightGBM,n_estimators: 50\nmax_depth: 4\nlearning_rate:...,n_estimators: 50\nmax_depth: 4\nlearning_rate:...,n_estimators: 50\nmax_depth: 10\nlearning_rate...,n_estimators: 50\nmax_depth: 10\nlearning_rate...
7,MLP,max_iter: 50\nlearning_rate_init: 0.1\nhidden_...,max_iter: 50\nlearning_rate_init: 0.1\nhidden_...,max_iter: 50\nlearning_rate_init: 0.01\nhidden...,max_iter: 50\nlearning_rate_init: 0.1\nhidden_...
8,RF,n_estimators: 400\nmin_samples_split: 12\nmin_...,n_estimators: 400\nmin_samples_split: 10\nmin_...,n_estimators: 600\nmin_samples_split: 10\nmin_...,n_estimators: 600\nmin_samples_split: 12\nmin_...
9,SVR,gamma: 1e-05\nC: 1,gamma: 1e-05\nC: 1,gamma: 1e-05\nC: 10,gamma: 1e-05\nC: 1
